# IBM Applied Data Science Capstone Course by Coursera



## Lusaka Schools


1. create a dataframe of neighborhoods in Lusaka, Lusaka using a csv file crated using data from wikipedia and http://www.mapcrow.info
2. Get the geographical coordinates of the schools
3. Obtain the school data for the neighborhoods from Foursquare API
4. Explore and cluster the neighborhoods


### 1. Import libraries

In [1]:
import requests 
from bs4 import BeautifulSoup
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy # uncomment this line if you haven't completed the Foursquare API lab
!pip install geocoder #Install geocoder


from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates



import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
#import pandas
import pandas as pd

# import k-means from clustering stage
from sklearn.cluster import KMeans


!pip install folium # uncomment this line if you haven't completed the Foursquare API lab

import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 102kB 6.5MB/s ta 0:00:011
     |████████████████████████████████| 102kB 7.9MB/s ta 0:00:011
Libraries imported.


### 2. Import CSV file and create a Dataframe for Lusaka Locations uploaded a the project assests

In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:

# Fetch the file
my_file = project.get_file("Lusaka_neighborhoods.csv")

# Read the CSV data file from the object storage into a pandas DataFrame
my_file.seek(0)
lsk_df=pd.read_csv(my_file, ) #nrows=10


In [4]:
lsk_df.head()


,Neighborhood
0,Avondale
1,Bauleni
2,Cathedrall Hill
3,Chainda
4,Chaisa


###  3. Get the geographical coordinates of Lusaka Neighborhoods

In [5]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize the variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Lusaka, Zambia'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [6]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in lsk_df["Neighborhood"].tolist() ]

In [7]:
coords

[[-15.42560999999995, 28.278710000000046],
 [-15.444569999999942, 28.377290000000073],
 [-15.42560999999995, 28.278710000000046],
 [-15.388319999999965, 28.40456000000006],
 [-15.384819999999934, 28.275150000000053],
 [-15.430420027860665, 28.307857296344118],
 [-15.358359999999948, 28.340620000000058],
 [-15.183329999999955, 30.233330000000024],
 [-15.455899999999929, 28.282480000000078],
 [-15.45528507564937, 28.354263431133557],
 [-15.38236999999998, 28.384190000000046],
 [-15.426259999999957, 28.272410000000036],
 [-15.44742999999994, 28.333950000000073],
 [-15.453149999999937, 28.335060000000055],
 [-15.357479999999953, 28.288010000000043],
 [-15.385609999999929, 28.306560000000047],
 [-15.361779999999953, 28.251750000000072],
 [-15.42560999999995, 28.278710000000046],
 [-15.391519999999957, 28.271520000000066],
 [-15.42560999999995, 28.278710000000046],
 [-15.350053831921423, 28.316828979442054],
 [-15.372319999999945, 28.234230000000025],
 [-15.392229999999927, 28.32041000000003

In [8]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [9]:
# merge the coordinates into the original dataframe
lsk_df['Latitude'] = df_coords['Latitude']
lsk_df['Longitude'] = df_coords['Longitude']

In [10]:
# check the neighborhoods and the coordinates
print(lsk_df.shape)
lsk_df.head()

(69, 3)


,Neighborhood,Latitude,Longitude
0,Avondale,-15.42561,28.27871
1,Bauleni,-15.44457,28.37729
2,Cathedrall Hill,-15.42561,28.27871
3,Chainda,-15.38832,28.40456
4,Chaisa,-15.38482,28.27515


In [11]:
#Correcting Avaondale Cordinates that were wrongly picked
lsk_df.at[0,'Latitude']= -15.382134
lsk_df.at[0,'Longitude']= 28.394314

In [12]:
print(lsk_df.shape)
lsk_df.head()


(69, 3)


,Neighborhood,Latitude,Longitude
0,Avondale,-15.382134,28.394314
1,Bauleni,-15.444570,28.377290
2,Cathedrall Hill,-15.425610,28.278710
3,Chainda,-15.388320,28.404560
4,Chaisa,-15.384820,28.275150


In [13]:
# save the DataFrame as CSV file
lsk_df.to_csv("lsk_df.csv", index=False)

### 4. Create a map of Lusaka with neighborhoods superimposed on top

In [14]:
# get the coordinates of Kuala Lumpur
address = 'Lusaka, Zambia'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Lusaka, Zambia {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Lusaka, Zambia -15.4164488, 28.2821535.


In [15]:
# create map of Toronto using latitude and longitude values
map_lsk = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(lsk_df['Latitude'], lsk_df['Longitude'], lsk_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_lsk)  
    


In [16]:
map_lsk

In [17]:
# save the map as HTML file
map_lsk.save('map_lsk.html')

### 5. Use the Foursquare API to Obatain Schools


In [18]:
# define Foursquare Credentials and Version

In [19]:
# The code was removed by Watson Studio for sharing.

In [20]:
limit=100
radius=2000

search_query='School'
column_names=['Neighborhood','Latitude','Longitude','School','School_Latitude','School_Longitude','School_category']
mydf=pd.DataFrame(columns=column_names)
for name, lat, lng in zip(lsk_df['Neighborhood'],lsk_df['Latitude'],lsk_df['Longitude']):
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, search_query, radius, limit)
    results = requests.get(url).json()
    k=results['response']['venues']
    for school in k:
        if len(school['categories'])>0:
            mydf=mydf.append({'Neighborhood':name,'Latitude':lat,'Longitude':lng,'School':school['name'],'School_Latitude':school['location']['lat'],'School_Longitude':school['location']['lng'],'School_category':school['categories'][0]['name']},ignore_index=True)
        else:
            mydf=mydf.append({'Neighborhood':name,'Latitude':lat,'Longitude':lng,'School':school['name'],'School_Latitude':school['location']['lat'],'School_Longitude':school['location']['lng'],'School_category':'Not Mentioned'},ignore_index=True)


In [21]:
mydf.head(30)


,Neighborhood,Latitude,Longitude,School,School_Latitude,School_Longitude,School_category
0,Avondale,-15.382134,28.394314,Joma Main Primary School,-15.366814,28.386275,Not Mentioned
1,Avondale,-15.382134,28.394314,Chelstone High School,-15.378919,28.375552,High School
2,Bauleni,-15.444570,28.377290,Kings high way school/Camp site,-15.456999,28.370786,Church
3,Cathedrall Hill,-15.425610,28.278710,Nkwazi Primary School,-15.418835,28.296261,Elementary School
4,Cathedrall Hill,-15.425610,28.278710,Metropolitan school,-15.444039,28.274386,Student Center
5,Cathedrall Hill,-15.425610,28.278710,St Thomas School,-15.406100,28.286620,University
6,Cathedrall Hill,-15.425610,28.278710,Nache Motor School,-15.407571,28.286461,Auto Dealership
7,Cathedrall Hill,-15.425610,28.278710,Nkhwazi primary school,-15.417904,28.297246,College Auditorium
8,Cathedrall Hill,-15.425610,28.278710,Dental Training School,-15.405850,28.283167,College Classroom
9,Chaisa,-15.384820,28.275150,Matero Boys Secondary School,-15.375098,28.275499,High School


## 5. Removing unneccessary rows and duplicates

In [22]:
mydf=mydf[~mydf.School.str.contains("Driving",case=False)]
mydf=mydf[~mydf.School.str.contains("Sound",case=False)]
mydf=mydf[~mydf.School.str.contains("Music",case=False)]
mydf=mydf[~mydf.School.str.contains("Management",case=False)]
mydf=mydf[~mydf.School.str.contains("office",case=False)]
mydf=mydf[~mydf.School.str.contains("bank",case=False)]
mydf=mydf[~mydf.School_category.str.contains("bank",case=False)]
mydf=mydf[~mydf.School_category.str.contains("architecture",case=False)]
mydf=mydf[~mydf.School_category.str.contains("engineering",case=False)]
mydf=mydf[~mydf.School_category.str.contains("library",case=False)]
#mydf=mydf.drop(21)
mydf=mydf.reset_index(drop=True)
mydf

,Neighborhood,Latitude,Longitude,School,School_Latitude,School_Longitude,School_category
0,Avondale,-15.382134,28.394314,Joma Main Primary School,-15.366814,28.386275,Not Mentioned
1,Avondale,-15.382134,28.394314,Chelstone High School,-15.378919,28.375552,High School
2,Bauleni,-15.444570,28.377290,Kings high way school/Camp site,-15.456999,28.370786,Church
3,Cathedrall Hill,-15.425610,28.278710,Nkwazi Primary School,-15.418835,28.296261,Elementary School
4,Cathedrall Hill,-15.425610,28.278710,Metropolitan school,-15.444039,28.274386,Student Center
5,Cathedrall Hill,-15.425610,28.278710,St Thomas School,-15.406100,28.286620,University
6,Cathedrall Hill,-15.425610,28.278710,Nache Motor School,-15.407571,28.286461,Auto Dealership
7,Cathedrall Hill,-15.425610,28.278710,Nkhwazi primary school,-15.417904,28.297246,College Auditorium
8,Cathedrall Hill,-15.425610,28.278710,Dental Training School,-15.405850,28.283167,College Classroom
9,Chaisa,-15.384820,28.275150,Matero Boys Secondary School,-15.375098,28.275499,High School


In [23]:
mydf1 = mydf.drop_duplicates(subset=['School_Latitude','School_Longitude'],
                                       keep='first').reset_index(drop=True)
mydf1.shape

(40, 7)

## 6. Categorizing the Data and Removing unneccessary categories

In [24]:
mydf1.loc[(mydf1.School.str.contains('Secondary',case=False)) | (mydf1.School.str.contains('High',case=False)) | (mydf1.School_category.str.contains('High',case=False)),'School_category']='High School'
mydf1.loc[(mydf1.School.str.contains('Play',case=False)) | (mydf1.School.str.contains('Primary',case=False)) | (mydf1.School_category.str.contains('elementary',case=False)) | (mydf1.School.str.contains('Kids',case=False)),'School_category']='Primary/Play'
mydf1.loc[mydf1.School_category.str.contains('college',case=False),'School_category']='Not Mentioned'
mydf1.loc[~((mydf1.School_category.str.contains('Not Mentioned',case=False))|(mydf1.School_category.str.contains('High School',case=False))|(mydf1.School_category.str.contains('Student Center',case=False))|mydf1.School_category.str.contains('Primary/Play',case=False)),'School_category']='Not Mentioned'
mydf1

,Neighborhood,Latitude,Longitude,School,School_Latitude,School_Longitude,School_category
0,Avondale,-15.382134,28.394314,Joma Main Primary School,-15.366814,28.386275,Primary/Play
1,Avondale,-15.382134,28.394314,Chelstone High School,-15.378919,28.375552,High School
2,Bauleni,-15.444570,28.377290,Kings high way school/Camp site,-15.456999,28.370786,High School
3,Cathedrall Hill,-15.425610,28.278710,Nkwazi Primary School,-15.418835,28.296261,Primary/Play
4,Cathedrall Hill,-15.425610,28.278710,Metropolitan school,-15.444039,28.274386,Student Center
5,Cathedrall Hill,-15.425610,28.278710,St Thomas School,-15.406100,28.286620,Not Mentioned
6,Cathedrall Hill,-15.425610,28.278710,Nache Motor School,-15.407571,28.286461,Not Mentioned
7,Cathedrall Hill,-15.425610,28.278710,Nkhwazi primary school,-15.417904,28.297246,Primary/Play
8,Cathedrall Hill,-15.425610,28.278710,Dental Training School,-15.405850,28.283167,Not Mentioned
9,Chaisa,-15.384820,28.275150,Matero Boys Secondary School,-15.375098,28.275499,High School


## 7.Plotting the Schools on Map using Folium

In [25]:
h_latitude=-15.40669
h_longitude=28.28713

school_map = folium.Map(location=[h_latitude, h_longitude], zoom_start=12) # generate map centred around the Conrad Hotel
# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(mydf1['Latitude'],mydf1['Longitude'],mydf1['School']):
    folium.Marker(
        [lat, lng],
        popup=label,
        icon=folium.Icon(color='red',icon='university', prefix='fa')
    ).add_to(school_map)

# display map
school_map

## 8. Getting Count of schools within 2KM in each neighborhood

In [26]:

counts_df=mydf.groupby('Neighborhood').count()
counts_df=counts_df.reset_index()
counts_df['Neighborhood']=counts_df['Neighborhood'].astype(str)
counts_df.shape

(56, 7)

### Counting categories

In [27]:
plt = mydf1['School_category'].value_counts().plot(kind='bar')

## 9. Getting the List of Neighbourhoods with no schools within 2km radius

In [28]:
diff=[]
flag=0
for i in lsk_df['Neighborhood']:
    for j in counts_df['Neighborhood']:
        if i==j:
            flag=1
            break
    if flag==0:
        diff.append(i)
    flag=0
print(len(diff))

13


In [29]:
latitudes=[]
longitudes=[]
count=0
# loop until you get the coordinates
for index,row in lsk_df.iterrows():
    lat_lng_coords = None
    neighborhood=row['Neighborhood']
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Lusaka, Zambia'.format(neighborhood))
        lat_lng_coords = g.latlng
    latitudes.append(lat_lng_coords[0])
    longitudes.append(lat_lng_coords[1])
lsk_df['Latitude']=latitudes
lsk_df['Longitude']=longitudes
lsk_df.head()

,Neighborhood,Latitude,Longitude
0,Avondale,-15.42561,28.27871
1,Bauleni,-15.44457,28.37729
2,Cathedrall Hill,-15.42561,28.27871
3,Chainda,-15.38832,28.40456
4,Chaisa,-15.38482,28.27515


In [30]:
#Correcting Avaondale Cordinates that were wrongly picked
lsk_df.at[0,'Latitude']= -15.382134
lsk_df.at[0,'Longitude']= 28.394314

## 10. Disance Closest to the Nearest Scho0l in Each neighborhood.

In [31]:
limit=100
radius=2000
r=[]
k=[]
count=0
for name, lat, lng in zip(lsk_df['Neighborhood'],lsk_df['Latitude'],lsk_df['Longitude']):
    while k==[] and count<10:
        count=count+1
        radius=radius + 100
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, search_query, radius, limit)
        results = requests.get(url).json()
        k=results['response']['venues']
    r.append(radius)
    k=[]
    count=0
    radius=2000
print(r)

[2100, 2100, 2100, 2600, 2100, 2100, 2100, 3000, 2100, 2100, 2100, 2100, 2100, 2100, 2100, 2100, 2500, 2100, 2100, 2100, 2100, 3000, 2100, 2100, 2100, 2400, 2100, 2700, 2100, 2100, 2100, 2100, 2100, 2100, 3000, 2100, 2100, 2100, 2100, 2100, 2600, 2100, 3000, 2100, 2100, 2100, 2100, 2100, 2100, 2100, 2100, 2100, 3000, 2100, 3000, 2100, 2100, 2100, 2100, 2100, 2100, 2100, 3000, 3000, 2100, 2100, 2100, 2100, 2100]


In [32]:
lsk_df['Near_school']=r
lsk_df.head()

,Neighborhood,Latitude,Longitude,Near_school
0,Avondale,-15.382134,28.394314,2100
1,Bauleni,-15.444570,28.377290,2100
2,Cathedrall Hill,-15.425610,28.278710,2100
3,Chainda,-15.388320,28.404560,2600
4,Chaisa,-15.384820,28.275150,2100


In [33]:
lsk_df['num_schools_in_2km']=0
lsk_df.head()

,Neighborhood,Latitude,Longitude,Near_school,num_schools_in_2km
0,Avondale,-15.382134,28.394314,2100,0
1,Bauleni,-15.444570,28.377290,2100,0
2,Cathedrall Hill,-15.425610,28.278710,2100,0
3,Chainda,-15.388320,28.404560,2600,0
4,Chaisa,-15.384820,28.275150,2100,0


In [34]:
lsk_df_copy=lsk_df
lsk_df_copy.shape

(69, 5)

## 11. K-Means to categorize the Neighbourhoods

In [35]:
k_means_df=lsk_df_copy[['Near_school','num_schools_in_2km']]
k_means_df.head()

,Near_school,num_schools_in_2km
0,2100,0
1,2100,0
2,2100,0
3,2600,0
4,2100,0


In [36]:
from sklearn.preprocessing import StandardScaler
X = k_means_df.values[:,:]
X = np.nan_to_num(X)
cluster_dataset = StandardScaler().fit_transform(X)
cluster_dataset

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([[-0.45629855,  0.        ],
       [-0.45629855,  0.        ],
       [-0.45629855,  0.        ],
       [ 1.20078567,  0.        ],
       [-0.45629855,  0.        ],
       [-0.45629855,  0.        ],
       [-0.45629855,  0.        ],
       [ 2.52645304,  0.        ],
       [-0.45629855,  0.        ],
       [-0.45629855,  0.        ],
       [-0.45629855,  0.        ],
       [-0.45629855,  0.        ],
       [-0.45629855,  0.        ],
       [-0.45629855,  0.        ],
       [-0.45629855,  0.        ],
       [-0.45629855,  0.        ],
       [ 0.86936882,  0.        ],
       [-0.45629855,  0.        ],
       [-0.45629855,  0.        ],
       [-0.45629855,  0.        ],
       [-0.45629855,  0.        ],
       [ 2.52645304,  0.        ],
       [-0.45629855,  0.        ],
       [-0.45629855,  0.        ],
       [-0.45629855,  0.        ],
       [ 0.53795198,  0.        ],
       [-0.45629855,  0.        ],
       [ 1.53220251,  0.        ],
       [-0.45629855,

In [37]:
cluster_dataset.shape

(69, 2)

In [38]:
from sklearn.cluster import KMeans 
num_clusters = 3

k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12)
k_means.fit(cluster_dataset)
labels = k_means.labels_

print(labels)

[0 0 0 2 0 0 0 1 0 0 0 0 0 0 0 0 2 0 0 0 0 1 0 0 0 2 0 2 0 0 0 0 0 0 1 0 0
 0 0 0 2 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0]


In [39]:
#lsk_df_copy=lsk_df_copy.drop(['label','label_4'],axis=1)
lsk_df_copy['label']=labels
lsk_df_copy.head(11)

,Neighborhood,Latitude,Longitude,Near_school,num_schools_in_2km,label
0,Avondale,-15.382134,28.394314,2100,0,0
1,Bauleni,-15.444570,28.377290,2100,0,0
2,Cathedrall Hill,-15.425610,28.278710,2100,0,0
3,Chainda,-15.388320,28.404560,2600,0,2
4,Chaisa,-15.384820,28.275150,2100,0,0
5,Chakunkula,-15.430420,28.307857,2100,0,0
6,Chamba Valley,-15.358360,28.340620,2100,0,0
7,Chandamali,-15.183330,30.233330,3000,0,1
8,Chawama,-15.455900,28.282480,2100,0,0
9,Chalala,-15.455285,28.354263,2100,0,0


In [40]:

label0_df=lsk_df_copy[lsk_df_copy['label']==0]
label1_df=lsk_df_copy[lsk_df_copy['label']==1]
label2_df=lsk_df_copy[lsk_df_copy['label']==2]
print(label0_df.shape,label1_df.shape,label2_df.shape)

(56, 6) (8, 6) (5, 6)


In [41]:
label0_df

,Neighborhood,Latitude,Longitude,Near_school,num_schools_in_2km,label
0,Avondale,-15.382134,28.394314,2100,0,0
1,Bauleni,-15.444570,28.377290,2100,0,0
2,Cathedrall Hill,-15.425610,28.278710,2100,0,0
4,Chaisa,-15.384820,28.275150,2100,0,0
5,Chakunkula,-15.430420,28.307857,2100,0,0
6,Chamba Valley,-15.358360,28.340620,2100,0,0
8,Chawama,-15.455900,28.282480,2100,0,0
9,Chalala,-15.455285,28.354263,2100,0,0
10,Chelston,-15.382370,28.384190,2100,0,0
11,Chibolya,-15.426260,28.272410,2100,0,0


In [42]:
label1_df

,Neighborhood,Latitude,Longitude,Near_school,num_schools_in_2km,label
7,Chandamali,-15.183330,30.233330,3000,0,1
21,George,-15.372320,28.234230,3000,0,1
34,Kamanga,-15.718580,28.584280,3000,0,1
42,Longacres,-15.416670,28.733330,3000,0,1
52,Mwambula,-15.533330,28.466670,3000,0,1
54,New Kasama,-15.470999,28.330371,3000,0,1
62,Roma,-15.383330,29.300000,3000,0,1
63,State Lodge,-15.448050,28.440500,3000,0,1


In [43]:
label2_df

,Neighborhood,Latitude,Longitude,Near_school,num_schools_in_2km,label
3,Chainda,-15.38832,28.40456,2600,0,2
16,Chunga,-15.36178,28.25175,2500,0,2
25,Industrial Area,-15.39689,28.25708,2400,0,2
27,John Howard,-15.46891,28.28364,2700,0,2
40,Lilanda,-15.37499,28.24362,2600,0,2


In [44]:
hyd_map = folium.Map(location=[h_latitude, h_longitude], zoom_start=12)
incidents = folium.map.FeatureGroup()
for lat, lng, in zip(label0_df['Latitude'],label0_df['Longitude']):
    incidents.add_child(
        folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
        )
    )
for lat, lng, in zip(label1_df['Latitude'],label1_df['Longitude']):
    incidents.add_child(
        folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        fill = True,
        fill_color='green',
        fill_opacity=0.6
        )
    )
for lat, lng, in zip(label2_df['Latitude'],label2_df['Longitude']):
    incidents.add_child(
        folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        fill = True,
        fill_color='red',
        fill_opacity=0.6
        )
    )
# add incidents to map
hyd_map.add_child(incidents)
hyd_map